In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn as nn
import torchvision.models as models
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch  
import numpy as np  
from PIL import Image  # For loading images  

class MoleCNN(nn.Module):
    def __init__(self, num_classes=1, pretrained_weights_path=None):
        super(MoleCNN, self).__init__()
        
        # Load ResNet model without downloading
        self.resnet = models.resnet50(weights=None)
        
        # If a path to pre-downloaded weights is provided, load them
        if pretrained_weights_path and os.path.exists(pretrained_weights_path):
            state_dict = torch.load(pretrained_weights_path)
            self.resnet.load_state_dict(state_dict)
        
        # Freeze all parameters in the pretrained model
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
        
        # Replace the final fully connected layer
        self.resnet.fc = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        # Get the features from ResNet
        x = self.resnet(x)
        
        return x




In [3]:
mole_model = MoleCNN()
mole_model.load_state_dict(torch.load('/kaggle/input/workingmodel/pytorch/default/1/mole_model_state_dict.pth', map_location=torch.device('cpu')))
mole_model.eval()

<ipython-input-3-7870409c943a>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mole_model.load_state_dict(torch.load('/kaggle/input/workingmodel/pytorch/default/1/mole_mode

MoleCNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [4]:
# Define the TestMoleDataset class
class TestMoleDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, img))]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # Open and convert to RGB

        if self.transform:
            image = self.transform(image)

        return image, img_path  # Return image path to track predictions

# Define the transformation for inference
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the dataset and dataloader
test_dataset = TestMoleDataset(image_dir="/kaggle/input/isic-2024-challenge/train-image/image", transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
from PIL import Image


In [9]:
# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model and move to device
model = MoleCNN(num_classes=1)  # Replace with your actual model class
model.load_state_dict(torch.load("/kaggle/input/workingmodel/pytorch/default/1/mole_model_state_dict.pth"))
model = model.to(device)  # Move model to device
model.eval()  # Set the model to evaluation mode

# Perform inference
predictions = []
image_paths = []
with torch.no_grad():  # Disable gradient computation
    for images, paths in test_loader:
        images = images.to(device)  # Move images to the same device as the model
        outputs = model(images)  # Forward pass
        probabilities = torch.sigmoid(outputs)  # Apply sigmoid for binary classification
        predictions.extend(probabilities.cpu().numpy())  # Move back to CPU for saving
        image_paths.extend(paths)  # Save corresponding image paths



<ipython-input-9-50907c02cc99>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/workingmodel/pytorch/default/1/mole_model_sta

In [11]:
import pandas as pd
import os

# Extract ISIC ID from image path
isic_ids = [os.path.basename(path).split(".")[0] for path in image_paths]

# Create a DataFrame in the correct format
results = pd.DataFrame({
    "isic_id": isic_ids,
    "target": [float(pred) for pred in predictions]  # Ensure float conversion
})

# Save to CSV
results.to_csv("/kaggle/working/train_predictions.csv", index=False)
print("Inference completed and predictions saved!")

<ipython-input-11-d3d186014169>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "target": [float(pred) for pred in predictions]  # Ensure float conversion


Inference completed and predictions saved!


In [12]:
results.head()

,isic_id,target
0,ISIC_9730802,0.438136
1,ISIC_2834883,0.412724
2,ISIC_5115027,0.448499
3,ISIC_3264822,0.399724
4,ISIC_1950204,0.463720
